In [96]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [97]:
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, PolynomialFeatures

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.metrics import mean_squared_error, precision_score, recall_score, f1_score, accuracy_score

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 75)

In [98]:
# Зафикcируем версию пакетов для воспроизводимости экспериментов
!pip freeze > requirements.txt

In [99]:
df_train = pd.read_csv('/kaggle/input/sf-dst-scoring/train.csv')
df_test = pd.read_csv('/kaggle/input/sf-dst-scoring/test.csv')
sample_submission = pd.read_csv('../input/sf-dst-scoring/sample_submission.csv')

In [100]:
# Добавим функцию для построения ROC-AUC curve
def roc_auc_curve(y_test, probs):
    fpr, tpr, threshold = roc_curve(y_test, probs)
    roc_auc = roc_auc_score(y_test, probs)

    plt.figure()
    plt.plot([0, 1], label='Baseline', linestyle='--')
    plt.plot(fpr, tpr, label = 'Regression')
    plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.legend(loc = 'lower right')
    plt.show()

In [101]:
# Добавим функцию для вывода на экран метрик качества
def print_metrics(y_test, y_pred):
    print(
        'Accuracy: {}'.format(np.round(accuracy_score(y_test, y_pred), 4)), 
        'f1_score: {}'.format(np.round(f1_score(y_test, y_pred), 4)),
        'Recall: {}'.format(np.round(recall_score(y_test, y_pred), 4)),
        'MSE: {}'.format(np.round(mean_squared_error(y_test, y_pred), 4)), sep="\n"
        )

In [102]:
# Просмотр результата тренировки модели с помощью Confusion Matrix - матрицы ошибок
def conf_matrix(y_test, y_pred):
    conf_mat = confusion_matrix(y_test, y_pred)
    sns.set(font_scale=1.4) 
    sns.heatmap(conf_mat, annot=True, annot_kws={"size": 16}, fmt='g', cmap='coolwarm')

In [103]:
# Добавим функцию для поиска выбросов
def outlier(col):
    IQR = df[col].quantile(0.75) - df[col].quantile(0.25)
    perc25 = df[col].quantile(0.25)
    perc75 = df[col].quantile(0.75)
    low_border = perc25 - 1.5 * IQR 
    high_border = perc75 + 1.5 * IQR
    print("Для {0} IQR: {1}, ".format(col,IQR),"Границы выбросов: [{0}, {1}].".format(low_border, high_border))
    print('Всего {} выбросов'.format(df[df[col] > high_border][col].count() + df[df[col] < low_border][col].count()))

In [104]:
df_train.info()

In [105]:
df_train.describe()

In [106]:
df_train.head()

In [107]:
df_test.info()

In [108]:
df_test.describe()

In [109]:
df_test.head()

In [110]:
sample_submission.info()

In [111]:
# в связи с тем, что количество строк df_test (36349 rows) и в df_train (73799 rows) 
# не соответсвует количеству строк в sample_submission (24354 rows) Объединяем тренировочную и тестовую выборки для обработки.
# Для последующего разделения создадим столбец ['train_test'], обозначим: 0 - train, 1 - test
df_train['train_test'] = 0
df_test['train_test'] = 1
df_test['default'] = 0 # В тестовой выборке отсутствует признак 'default' - его необходимо предсказать, поэтому заполним его нулями
df = pd.concat([df_train, df_test])
df.info()

In [112]:
df.shape

In [113]:
df.head()

Объединенная таблица имеет количество строк(110148 rows), равное сумме строк тренировочной и тестовой выборок  

In [114]:
#Посмотрим на распределение целевой переменной и вычислим процент default-клиентов
print(df['default'].value_counts(), df['default'].value_counts()[1]/len(df.default)*100)

In [115]:
# Проверим наличие пропусков в данных
df.isna().sum()

Видим, что пропуски имеются только в столбце 'education'

In [116]:
#Посчитаем количество пропусков и уникальных значений в столбце 'education'
df['education'].value_counts(dropna=False)

In [117]:
# Посмотрим на распределение признака 'education' на гистограмме
df['education'].hist()

In [118]:
# предположим, что пропуски в признаке "education' можнозаменить самым встречающимся значением - SCH
df['education'][df.education.isna()] = df['education'].mode()[0]

In [119]:
# Проверим заново наличие пропусков в признаке 'education'
df['education'].isna().sum()

In [120]:
# Преобразуем столбец 'app_date' в формат datetime
df['app_date'] = pd.to_datetime(df['app_date'])
print(f"Год и количество обращений: \n{df['app_date'].dt.year.value_counts()}\nМесяц и количество обращений: \n{df['app_date'].dt.month.value_counts()}") 

Все обращения приходятся на 2014 год в интервале 4 месяцев - с января по апрель

In [121]:
#Создаем новые признаки:
df['app_day'] = df['app_date'].dt.day
df['app_month'] = df['app_date'].dt.month
df['app_weekday'] = df['app_date'].dt.weekday

print(f"Дни недели и количество обращений: \n{df['app_weekday'].value_counts().sort_values(ascending=True)}")

Исходя из полученных резудьтатов, видно, что на выходные дни приходится значительно меньше обращений, чем на будние дни недели

In [122]:
# Удалим столбец 'app_date' за ненадобностью
df.drop(['app_date'], axis=1, inplace=True)

In [123]:
# Посмотрим гистограммы по новым признакам
for col in df.iloc[:,19:22].columns:
    sns.countplot(x = df[col], data = df)
    plt.show()

# Разделим признаки на группы по типу 

In [124]:
df.info()

In [125]:
#бинарнные признаки (переменные)
bin_cols = [
    'sex', 'car', 'car_type', 'good_work', 'foreign_passport'
]

#категориальные признаки
cat_cols = [
    'education', 'home_address', 'work_address', 'sna', 
    'first_time','app_month', 'app_weekday'
]

#Числовые признаки
num_cols = [
    'age', 'decline_app_cnt', 'score_bki', 'bki_request_cnt', 
    'region_rating', 'income', 'app_day'
]

# Построим наивную модель

In [126]:
#Посмотрим наивную модель, для чего используем LabelEncoder()
label_encoder = LabelEncoder()

In [127]:
#Преобразуем бинарные признаки
for col in bin_cols:
    df[col] = label_encoder.fit_transform(df[col])

In [128]:
#Перекодируем признак 'education'
education_dict = {'SCH':0, 'UGR':1, 'GRD':2, 'PGR':3, 'ACD':4}
df['education'] = df['education'].replace(education_dict)

In [129]:
df.head()

In [130]:
df.info()

In [131]:
#Разделим датасет на тренировочную и тестовую выборки по метке в столбце 'train_test'
df_train_naiv = df.query('train_test == 0').drop(['train_test'], axis=1)
df_test_naiv = df.query('train_test == 1').drop(['train_test'], axis=1)

In [132]:
X = df_train_naiv.drop(['default'], axis=1).values
y = df_train_naiv['default'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [133]:
model_naiv = LogisticRegression(max_iter=1000)
model_naiv.fit(X_train, y_train)
y_pred = model_naiv.predict(X_test)

In [134]:
probs = model_naiv.predict_proba(X_test)

In [135]:
probs = probs[:,1]
roc_auc_curve(y_test, probs)

In [136]:
print_metrics(y_test, y_pred)

In [137]:
conf_matrix(y_test, y_pred)

исходя из матрицы ошибок, мы видим, что наивная модель не определяет default-пользователей!!!

# Улучшение модели

## Анализируем числовые признаки

In [138]:
df.info()

In [139]:
# Построим гистограммы распределения числовых величин
for i in num_cols:
    plt.figure(figsize=(12, 6))
    sns.distplot(df[i][df[i] > 0].dropna(), kde = False, rug=False)
    plt.title(i)
    plt.show()

In [140]:
 #Посмотрим на боксплоты и выбросы
for col in num_cols:
    plt.figure(figsize=(15,3))
    sns.boxplot(y = df['default'], x = df[col], showmeans=True, meanline=True,  orient='h')

Из боксплотов мы видим, что default-пользователи:
- немного моложе
- по ним имеется больше запросов в БКИ
- имеется больше отказов по обращениям
- имеют меньший доход

In [141]:
#определим количество выбросов для числовых признаков
for col in num_cols:
    outlier(col)

Переведем признаки 'decline_app_cnt' и 'bki_request_cnt' в категориальные по принципу 1-2-3-много

In [142]:
# Заменим значения в признаке 'decline_app_cnt'
df['decline_app_cnt'] = df['decline_app_cnt'].apply(lambda x: (x if x <= 4 else 5))
# Одновременно удалим столбец из числовых признаков и добавим в категориальные
num_cols.remove('decline_app_cnt')
cat_cols.append('decline_app_cnt')

In [143]:
#Заменим занчения в признаке 'bki_request_cnt'
df['bki_request_cnt'] = df['bki_request_cnt'].apply(lambda x: (x if x <= 8 else 9))
# Одновременно удалим столбец из числовых признаков и добавим в категориальные
num_cols.remove('bki_request_cnt')
cat_cols.append('bki_request_cnt')

Посмотрим на распределение признаков в тренировочной и тестовой выборках

In [144]:
# для тренировочной выборки
df[df['train_test'] == 0][num_cols].hist(figsize=(40, 30), bins=50)


In [145]:
#Для тестовой выборки
df[df['train_test'] == 1][num_cols].hist(figsize=(40, 30), bins=50)

Видим, что распределение числовых признаков в тренировочной и тестовой выборках одинаковое, но по некоторым признакам имеется существенное смещение влево

In [146]:
# Построим матрицу корреляции для числовых признаков
correlation_num_cols = df[num_cols].corr()
sns.heatmap(correlation_num_cols.abs(), vmin=0, vmax=1, annot = True, fmt=".2f", cmap='coolwarm')

Видим, что корреляционные зависимости между числовыми признаками незначительные

In [147]:
# Проведем оценку значимости числовых признаков с помощью функции f_classif
# Визуализируем результат
important_num = pd.Series(f_classif(df[num_cols], df['default'])[0], index = num_cols)
important_num.sort_values(inplace = True)
important_num.plot(kind = 'barh', figsize=(10, 6))

видим, что самый значимый признак из числовых признаков является 'score_bki'

In [148]:
df[num_cols].describe()

Прологарифмируем значения столбцов со смещенным распределением

In [149]:
df['income'] = df['income'].apply(lambda x: np.log(x + 1))
#df['score_bki'] = df['score_bki'].apply(lambda x: np.log(x + 1))

In [150]:
# Построим гистограммы логарифмированных данных
for i in num_cols:
    plt.figure(figsize=(12, 6))
    sns.distplot(df[i][df[i] > 0].dropna(), kde = False, rug=False)
    plt.title(i)
    plt.show()

Нормальное распределение признака 'income' +
отказываемся от логарифмирования признака 'score_bki'
и видим, что от дня обращения не зависит целевая переменная default, поэтому удалим признак 'app_day'

In [151]:
df.drop(['app_day'], axis=1, inplace=True)
num_cols.remove('app_day')

для прологарифмированных и преобразованных данных построим боксплоты и определим выбросы

In [152]:
for col in num_cols:
    plt.figure(figsize=(15,3))
    sns.boxplot(y = df['default'], x = df[col], showmeans=True, meanline=True, orient = 'h')

In [153]:
for col in num_cols:
    outlier(col)

# Анализируем категориальные признаки

In [154]:
print(cat_cols)

In [155]:
df[cat_cols].info()

In [156]:
for col in cat_cols:
    print(df[col].value_counts())

In [157]:
for col in cat_cols:
    print(df[col].value_counts()/len(df) * 100)

In [158]:
for col in cat_cols:
    sns.barplot(x=col, y='default', data=df[[col, 'default']])
    plt.show()

In [159]:
df_dum = pd.get_dummies(df[cat_cols].astype('object'))
df_dum

In [160]:
df_dum.info()

In [161]:
df = pd.concat([df, df_dum], axis=1)

In [162]:
df.info()

In [163]:
cat_cols = df_dum.columns
print(cat_cols)

# Проанализируем бинарные признаки

In [164]:
print(bin_cols)

In [165]:
for col in bin_cols:
    print(df[col].value_counts())

In [166]:
#Посмотрим на распределение бинарных признаков
df[bin_cols].hist(bins=2, figsize=(20, 10))

In [167]:
df[bin_cols].describe()

In [168]:
# оценка значимости бинарных признаков:
important_bin_cols = pd.Series(mutual_info_classif(df[bin_cols], df['default'],
                                        discrete_features=True), index=bin_cols)
important_bin_cols.sort_values(inplace=True)
important_bin_cols.plot(kind='barh', figsize=(20,10), colormap='gist_gray')

Исходя из оценки значимости бинарных признаков, можно сделать вывод, что наиболее значимыми бинарными признаками являются 'foreign_passport' и 'car_type'

# Модель

# 1). Logistic Regression

In [169]:
#Разделим датасет на тренировочную и тестовую выборки по метке в столбце 'train_test'
df_train_model = df.query('train_test == 0').drop(['train_test'], axis=1)
df_test_model = df.query('train_test == 1').drop(['train_test'], axis=1)

In [170]:
X = df_train_model.drop(columns=['client_id', 'default']).values
y = df_train_model['default'].values

In [171]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [172]:
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

In [173]:
probs = model.predict_proba(X_test)
probs = probs[:,1]

In [174]:
roc_auc_curve(y_test, probs)

In [175]:
y_pred = model.predict(X_test)
pd.Series(y_pred).value_counts()

In [176]:
print_metrics(y_test, y_pred)

In [177]:
conf_matrix(y_test, y_pred)

Модель предсказывает default-клиентов, однако количество верно предсказанных значений крайне низко, при этом ошибка второго рода очень велика (1783 неверных предсказаний - предсказан возврат кредита, при случившемся в реальности дефолте).
Это происходит скорее всего из-за несбалансированности целевой переменной

# 2). Logistic Regression (class_weight = balanced)

Попробуем сбалансировать классы при помощи параметра class_weight=balanced

In [178]:
model = LogisticRegression(class_weight = 'balanced',max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
probs = model.predict_proba(X_test)
probs = probs[:,1]

In [179]:
pd.Series(y_pred).value_counts()

In [180]:
roc_auc_curve(y_test, probs)

In [181]:
print_metrics(y_test, y_pred)

In [182]:
conf_matrix(y_test, y_pred)

Несмотря на тот факт, что AUC=0.744 и не изменился по сравнению с предыдущей моделью, метрики значительно увеличились, а также значительно увеличилось количество предсказаний целевой переменной, что видно в матрице ошибок - в то же время остается высокой ошибка первого рода - 4277 неверных предсказаний целевой переменной (то есть предсказан дефолт, а кредит в действительности возвращен).
Таким образом, сбалансированность целевой переменной позволило улучшить модель. Оставляем пока его

# 3) GridSearchCV

Попробуем применить гиперпараметры

In [183]:
# Добавим типы регуляризации
#penalty = ['l1', 'l2']
# Зададим ограничения для параметра регуляризации
#C = np.logspace(0, 4, 10)
# Создадим гиперпараметры
#hyperparameters = dict(C=C, penalty=penalty)

In [184]:
#clf = GridSearchCV(model, hyperparameters, cv=5)
#best_model = clf.fit(X_train, y_train)

#print('Best penalty=', best_model.best_estimator_.get_params()['penalty'])
#print('Best C=', best_model.best_estimator_.get_params()['C'])

In [185]:
#model = LogisticRegression(penalty = 'l2', class_weight='balanced', max_iter=1000, solver='liblinear', C=2.78)
#model.fit(X_train, y_train)
#y_pred = model.predict(X_test)

In [186]:
#probs = model.predict_proba(X_test)
#probs = probs[:,1]

In [187]:
#roc_auc_curve(y_test, probs)

In [188]:
#print_metrics(y_test, y_pred)

In [189]:
#conf_matrix(y_test, y_pred)

Видим, что применение гиперпараметров не позволило улучшить модель - отказываемся от нее

# Submission

Предсказываем целевую переменную на данных из тестовой выборки

In [191]:
df_test_model.head(4)

In [194]:
df_test_model.columns

In [195]:
df_test_model.info()

In [196]:
X_pred = df_test_model.drop(['client_id', 'default'], axis=1).values
target_pred = model.predict_proba(X_pred)[:,1]
submission = pd.concat([df_test_model.client_id, pd.Series(target_pred, name='default')], axis=1)
submission.to_csv('submission.csv', index=False)
submission.shape

In [197]:
submission.info()

In [198]:
submission.sample(10)